# visualizer.ipynb

In [1]:
import sqlite3
import cv2
import pandas as pd
import numpy as np

In [2]:
def annotate_image_with_polygon_and_rectangle(original_image_path, x_min, y_min, x_max, y_max, polygon_points, color):
    # Load original image
    img_original = cv2.imread(original_image_path).copy()
    
    # Overlay detected object with a semitransparent polygon mask
    img_overlay = np.zeros_like(img_original)
    # mask_color = (0, 0, 255)  # Red color in BGR
    alpha = 0.4  # Transparency factor (0.0 = transparent, 1.0 = opaque)
    img_overlay = cv2.fillPoly(img_overlay, polygon_points, color)
    img_annotated = cv2.addWeighted(img_overlay, alpha, img_original, 1 - alpha, 0)

    # Draw rectangle
    start_point = (x_min, y_min)
    end_point = (x_max, y_max)
    thickness = 1
    cv2.rectangle(img_annotated, start_point, end_point, color, thickness)
    
    return img_annotated

In [3]:
database_path = 'sam3_detections.sqlite3'
sql = 'select * from detections order by image_path, confidence desc;'

conn = sqlite3.connect(database_path)
df = pd.read_sql_query(sql, conn)
conn.close()

for i, r in df.iterrows():
    original_image_path = r['image_path']
    x_min = int(r['x_min'])
    y_min = int(r['y_min'])
    x_max = int(r['x_max'])
    y_max = int(r['y_max'])
    polygon_points_str = r['polygon_points_str']
    polygon_points = np.array(eval(polygon_points_str), np.int32)          
    polygon_points = [polygon_points.reshape((-1, 1, 2))]
    color = (0, 0, 255)  # Red color in BGR
    annotated_image_path = f'{original_image_path}-{i:02d}.jpg'    
    img_annotated = annotate_image_with_polygon_and_rectangle(original_image_path, x_min, y_min, x_max, y_max, polygon_points, color)
    cv2.imwrite(annotated_image_path, img_annotated)
    
print('FINISHED')

FINISHED
